# Disable Rounding of Some Transfers


When transfers are calculated, in some cases intermediate results or the final amount need to be rounded based on German law. As a default, GETTSIM also rounds these results. However, we can disable rounding which is for example beneficial if results are used within a numerical optimization. This HowTo describes how you specify if rounding should be applied or not. At the end, some information are given how rounding is implemented at the pre-implemented functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gettsim import set_up_policy_environment
from gettsim import compute_taxes_and_transfers
from gettsim.synthetic_data.synthetic import create_synthetic_data

## Create synthetic data set and set up policy environment

In [ ]:
## idea for use of synthetical data 
data = create_synthetic_data(
    hh_typen=["single"], 
     n_children=[0], 
     age_adults=[70], 
    heterogeneous_vars={"bruttolohn_m": np.arange(0, 1000, 1)}
)
data["rentner"] = True
data["entgeltpunkte"] = np.arange(10, 10.1, 0.0001)
data["entgeltp_grundr"] = data["entgeltpunkte"]
data.head()

In [ ]:
# Set up policy environment
policy_params, policy_functions = set_up_policy_environment(date=2021)


## Compute Grundrentenzuschlag and Staatliche Rente including rounding 

The default is that all outputs that are rounded according to German law are also rounded in GETTSIM. 

One of these variables is the Grundrentenzuschlag (`grundr_zuschlag_m`) which is rounded to two decimal points.

In [ ]:
results = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets=["grundr_zuschlag_m", "staatl_rente_m"],
    functions=policy_functions,
)
results.head()

When zooming in, rounding leads to a step function which is very hard to optimize over.

In [ ]:
plt.plot("entgeltpunkte", "grundr_zuschlag_m", data=results.join(data).query("entgeltpunkte < 10.01"));

## Disable rounding 

Disabling rounding leads to a smooth relation between `entgeltpunkte` and `grundr_zuschlag_m`

In [ ]:
results = compute_taxes_and_transfers(
    data=data,
    params=policy_params,
    targets=["grundr_zuschlag_m", "staatl_rente_m"],
    functions=policy_functions,
    rounding=False
)
results.head()

In [ ]:
plt.plot("entgeltpunkte", "grundr_zuschlag_m", data=results.join(data).query("entgeltpunkte < 10.01"));

## Implementation

This subsection gives an impression how it is specified within GETTSIM which functions should be rounded and in which way. For the common user, this subsection can be ignored if you do not plan to implement your own policy functions.

Whether a function should be rounded and in which way, is added to the function as attribute.

In [ ]:
from gettsim.transfers.grundrente import grundr_zuschlag_m
grundr_zuschlag_m.__roundingspec__

To add these attributes to functions you have written yourself (see [policy functions tutorial](policy_functions.ipynb)), you can use the function `add_rounding_spec` as decorator as follows:

In [ ]:
from gettsim.shared import add_rounding_spec

@add_rounding_spec(base=0.01, direction="up")
def your_own_function():
    out = 0.2
    return out

your_own_function.__roundingspec__

`direction` needs to be one of 'up', 'down', or 'nearest'

In [ ]:
from gettsim.shared import add_rounding_spec

@add_rounding_spec(base=0.01, direction="upper")
def your_own_function():
    out = 0.2
    return out

your_own_function.__roundingspec__